In [1]:
{
  "cells": [],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.9.0"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}

{'cells': [],
 'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'language_info': {'name': 'python', 'version': '3.9.0'}},
 'nbformat': 4,
 'nbformat_minor': 5}

In [2]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


封装TIMIT数据集

In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int64)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

分成训练和测试数据集

In [4]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


In [5]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

模型

In [6]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.layer1 = nn.Linear(429,1024)
        self.layer2 = nn.Linear(1024,512)
        self.layer3 = nn.Linear(512,128)
        self.out = nn.Linear(128,39)
        
        self.dropout = nn.Dropout(p=0.5)
        
        self.act_fn = nn.ReLU()
        
    def forward(self,x):
        x = self.layer1(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer2(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        
        x = self.layer3(x)
        x = self.act_fn(x)
        
        x = self.out(x)
        
        return x

In [7]:
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
same_seeds(0)

device = get_device()
print('Device:', device)

# learning parameters
num_epoch = 20
learning_rate = 0.0001

model_path = './model.pt'

#the model we use to train
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    val_acc = 0.0
    train_loss = 0.0
    val_loss = 0.0
    
    # training
    model.train()
    for i,data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _,train_pred = torch.max(outputs,1)
        batch_loss.backward()
        optimizer.step()

        train_loss += batch_loss.item()
        train_acc += (train_pred == labels).sum().item()

    # validation
    model.eval()
    with torch.no_grad():
        for i,data in enumerate(val_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            _,val_pred = torch.max(outputs,1)

            val_loss += batch_loss.item()
            val_acc += (val_pred == labels).sum().item()

    print('[Epoch {}] Train Loss: {:.4f} Acc: {:.4f} | Val Loss: {:.4f} Acc: {:.4f}'.format(
        epoch+1, train_loss/len(train_loader), train_acc/len(train_loader),
        val_loss/len(val_loader), val_acc/len(val_loader)))
    # save the best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)

if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('No validation set, model saved.')
    
    
    
    
# testing
test_set = TIMITDataset(test)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))


# predict test set
predict = []
model.eval()
with torch.no_grad():
    for i,data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _,test_pred = torch.max(outputs,1)
        
        for y in test_pred.cpu().numpy():
            predict.append(y)


Device: cuda
[Epoch 1] Train Loss: 1.5064 Acc: 34.7386 | Val Loss: 1.1359 Acc: 41.4355
[Epoch 2] Train Loss: 1.2720 Acc: 38.6786 | Val Loss: 1.0432 Acc: 43.0757
[Epoch 3] Train Loss: 1.2058 Acc: 39.8321 | Val Loss: 0.9991 Acc: 43.8455
[Epoch 4] Train Loss: 1.1634 Acc: 40.5512 | Val Loss: 0.9626 Acc: 44.3668
[Epoch 5] Train Loss: 1.1335 Acc: 41.0740 | Val Loss: 0.9455 Acc: 44.6264
[Epoch 6] Train Loss: 1.1104 Acc: 41.4273 | Val Loss: 0.9236 Acc: 45.0578
[Epoch 7] Train Loss: 1.0930 Acc: 41.7460 | Val Loss: 0.9149 Acc: 45.1774
[Epoch 8] Train Loss: 1.0790 Acc: 42.0144 | Val Loss: 0.8997 Acc: 45.4579
[Epoch 9] Train Loss: 1.0633 Acc: 42.3103 | Val Loss: 0.8893 Acc: 45.5666
[Epoch 10] Train Loss: 1.0542 Acc: 42.4114 | Val Loss: 0.8873 Acc: 45.6085
[Epoch 11] Train Loss: 1.0446 Acc: 42.6260 | Val Loss: 0.8762 Acc: 45.9194
[Epoch 12] Train Loss: 1.0348 Acc: 42.7638 | Val Loss: 0.8695 Acc: 45.9368
[Epoch 13] Train Loss: 1.0276 Acc: 42.8922 | Val Loss: 0.8661 Acc: 46.0341
[Epoch 14] Train Loss

TypeError: DataLoader.__init__() got an unexpected keyword argument 'weight_decay'

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

1988